# Money, death and t



We return, again, to the death penalty.

In [ ]:
# Array library.
import numpy as np
# Data frames library.
import pandas as pd
# Turn on a setting to use Pandas more safely.
# We will discuss this setting later.
pd.set_option('mode.chained_assignment', 'raise')
# Set up plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

# Load the OKpy test library and tests.
from client.api.notebook import Notebook
ok = Notebook('money_t.ok')

As revision, we are going to do a permutation test on the results from a
sample of the US [General Social Survey](http://www.gss.norc.org) from 2002.

See the `money_death` exercise for a longer form.  Here we jog through the steps to get the data we need.

In [ ]:
# Run this cell.
# Read the data into a data frame
# One row per respondent.
gss = pd.read_csv('GSS2002.csv')

# Recode the income column.
def recode_income(value):
    if pd.isna(value):  # Missing value
        return np.nan
    if value == 'under 1000':
        return 500
    low_str, high_str = value.split('-')
    low, high = int(low_str), int(high_str)
    return np.mean([low, high])

# Get the columns of interest into their own data frame, and recode.
money_death = pd.DataFrame()
money_death['Income'] = gss['Income'].apply(recode_income)
money_death['DeathPenalty'] = gss['DeathPenalty'].copy()
# Drop all missing values.
slim_md = money_death.dropna()
# Show the first five rows.
slim_md.head()

Next fetch the income values for those in "Favor" of the death penalty, and for those who "Oppose".  Put them in arrays.

In [ ]:
# Run this cell
income = slim_md['Income']
death = slim_md['DeathPenalty']
favor_income_arr = np.array(income[death == 'Favor'])
oppose_income_arr = np.array(income[death == 'Oppose'])

In [ ]:
# Show a histogram of the in-favor income values.
plt.hist(favor_income_arr)

In [ ]:
# Show a histogram of the oppose income values.
plt.hist(oppose_income_arr)

Calculate the difference in mean income between the groups.  This is the
difference we observe.

In [ ]:
actual_diff = ...
actual_diff

In [ ]:
_ = ok.grade('q_8_actual_diff')

Next, the permutation test.

In [ ]:
n_favor = ...

In [ ]:
fake_differences = np.zeros(10000)
for i in np.arange(10000):
    # Permute the pooled incomes
    shuffled = np.random.permutation(pooled)
    # Make a fake favor sample
    fake_favor = shuffled[:n_favor]
    # Make a fake opposed sample
    fake_oppose = shuffled[n_favor:]
    # Calculate the mean difference for the fake samples
    fake_diff = np.mean(fake_favor) - np.mean(fake_oppose)
    # Put the mean difference into the fake_differences array.
    fake_differences[i] = fake_diff
# Show the first 10 fake differences
fake_differences[:10]

Do a histogram of the fake differences.  This is the *sampling distribution*
of the *difference in means* in the ideal world (*null hypothesis*) of no
difference in the income values in the underlying population.

In [ ]:
#- Do a histogram of the sampling distribution.
...

Show the proportion of the differences you calculated that were greater than or
equal to the actual difference.

In [ ]:
p_fake_ge_actual = ...
p_fake_ge_actual

In [ ]:
_ = ok.grade('q_11_p_fake_ge_actual')

This proportion gives an estimate of the probability of seeing a difference
this large, if the incomes all come from the same underlying population.

## Compare to t

Next calculate the permutation equivalent of the t-statistic.  This is the actual difference in means that you have above, divided by the standard deviation (`np.std`) of the sampling distribution.  Call permutation t equivalent `like_t`.

In [ ]:
like_t = ...
# Show the result
like_t

In [ ]:
_ = ok.grade('q_like_t')

Now calculate the actual t statistic.  Follow the recipe in the [permtuation
and t test page](https://matthew-brett.github.io/cfd2020/permutation/permutation_and_t_test), to
import the t-test routine from Scipy, and run it on the two arrays of values.

In [ ]:
#- Import the scipy t-test routine.
from scipy... import ...
#- Calculate the t result.
t_result = ...
#- Get the t-statistic from the result
t_stat = ...
# Show the t statistic as calculated by Scipy
t_stat

In [ ]:
_ = ok.grade('q_t_stat')

The next step is to calculate the t-test t value step by step.

Remember, the t test has a numerator that is the actual difference in means,
divided by a denominator, which is an *estimate*, using some assumptions, of
the *standard deviation of the sampling distribution*.

The *t denominator* comprises two components.  The first we will calculate
further below; it is an estimate of the variation left after subtracting the
two group means.  The second is a bit arcane, and I've just given that part to
you here:

In [ ]:
# The number of respondents in the in-favor group.
n1 = len(favor_income_arr)
# The number of respondents in the oppose group.
n2 = len(oppose_income_arr)
# The second part of the denominator.
second_part_of_denom = np.sqrt(1 / n1 + 1 / n2)
second_part_of_denom

Now for the first part.  To calculate it, you will need something called
"degrees of freedom".  It's not worth going into what that means here, but here
are the degrees of freedom in this case:

In [ ]:
# Number of respondents in both groups, minus the number of means we use
# for the subtractions below.
degrees_of_freedom = n1 + n2 - 2
degrees_of_freedom

Now to finish the calculation of the first half of the denominator, do the
following steps.  See the [permtuation and t test page](https://matthew-brett.github.io/cfd2020/permutation/permutation_and_t_test), for inspiration.

In [ ]:
#- Subtract the mean of the "favor" incomes from the "favor_income_arr" to
#- give "mc_favor_incomes" (mc stands for mean-centered).
mc_favor_incomes = ...
#- Subtract the mean of the "oppose" incomes from the "oppose_income_arr" to
#- give "mc_oppose_incomes".
mc_oppose_incomes = ...
#- Pool the two mc arrays and square each value.
mc_pooled_squared = ...
#- Sum the pooled squared values, divide by the degrees of freedom and take
#- the square root of the result, to give the missing component of the
#- denominator.
first_part_of_denom = ...
# Show the result.
first_part_of_denom

In [ ]:
_ = ok.grade('q_first_part_of_denom')

The full denominator is the two parts, multiplied.  Remember that the t
denominator is an estimate, using some assumptions, of the standard deviation
of the sampling distribution.  Your permutation test also estimated the
sampling distribution.  In the cell below you should see that the t denominator
estimate and the permutation group estimate are pretty similar:

In [ ]:
# Run this cell
t_denom = first_part_of_denom * second_part_of_denom
print('Standard deviation of sampling distribution')
print('Permutation estimate', np.std(fake_differences))
print('t-test estimate', t_denom)

For reflection: why might these two estimates be different?

Finally, calculate the t-test t value - the numerator (observed difference)
divided by the denominator (estimate of standard deviation of observed
difference).

In [ ]:
#- Divide the numerator by the denominator to get "calculated_t"
calculated_t = ...
# Show the result
calculated_t

You should find this is the same (to many decimal places) as the result that
Scipy calculated above.

In [ ]:
_ = ok.grade('q_calculated_t')

Finally, we can use a function in Scipy to give a probability value to the t
value.  Don't worry about remembering this function - we won't use it again in
this course.

In [ ]:
# Show the t-test probability value
# The Scipy statistics sub-module.
import scipy.stats as sps
# Using the Cumulative Distribution Function method of the t distribution, to
# calculate the one-tailed p-value for the t test.
t_p_value = 1 - sps.t.cdf(calculated_t, degrees_of_freedom)
print('p value for t test', t_p_value)

This will be similar to the p value estimate from the permutation test, that
you calculated above.

In [ ]:
print('p value for permutation test', p_fake_ge_actual)

For reflection: why might these two be different?

## Done

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]